In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import glob
import seaborn as sns
import plotly.express as px
from tqdm import tqdm
import time
import re
from IPython.core.display import display, HTML
display(HTML('<style>.container { width:100% !important; }</style>'))

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 300)

C:\Users\matth\AppData\Local\Temp\ipykernel_18184\1510210558.py:10: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
files = pd.concat(
    map(pd.read_excel, glob.glob("well_data/20*.xlsx"))
)
files.to_csv("ND_well_data_2015_2023.csv", index=False)

In [3]:
oil = pd.read_csv("ND_well_data_2015_2023.csv")

oil.columns = [name.lower() for name in oil.columns]

oil["company"] = oil['company'].str.replace(',',"", regex=False)
oil["company"] = oil['company'].str.replace('.',"", regex=False)

In [4]:
# Prima
index =[949859,931153]
oil['quarter'].iloc[index]='SWSE'
oil['section'].iloc[index]=11.0
oil['township'].iloc[index]=146.0
oil['range'].iloc[index]=99.0
# sinclair
index=[951247]
oil['quarter'].iloc[index]='NENE'
oil['section'].iloc[index]=9.0
oil['township'].iloc[index]=153.0
oil['range'].iloc[index]=93.0

C:\Users\matth\AppData\Local\Temp\ipykernel_18184\1044223139.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oil['quarter'].iloc[index]='SWSE'
C:\Users\matth\AppData\Local\Temp\ipykernel_18184\1044223139.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oil['section'].iloc[index]=11.0
C:\Users\matth\AppData\Local\Temp\ipykernel_18184\1044223139.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oil['township'].iloc[index]=146.0


In [5]:
county_abbv = {
    "BIL":"Billings",
    "BOT":"Bottineau",
    "BOW":"Bowman",
    "BRK":"Burke",
    "DIV":"Divide",
    "DUN":"Dunn",
    "GV":"Golden Valley",
    "MCH":"McHenry",
    "MCK":"McKenzie",
    "MCL":"McLean",
    "MTL":"Mountrail",
    "REN":"Renville",
    "SLP":"Slope",
    "STK":"Stark",
    "WRD":"Ward",
    "WIL":"Williams"
}

oil['county_name'] = oil['county'].map(county_abbv)

In [6]:
oil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1606028 entries, 0 to 1606027
Data columns (total 22 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   reportdate   1606028 non-null  object 
 1   api_wellno   1606028 non-null  int64  
 2   fileno       1606028 non-null  int64  
 3   company      1606028 non-null  object 
 4   wellname     1606028 non-null  object 
 5   quarter      1605775 non-null  object 
 6   section      1606004 non-null  float64
 7   township     1606004 non-null  float64
 8   range        1606004 non-null  float64
 9   county       1606028 non-null  object 
 10  fieldname    1606028 non-null  object 
 11  pool         1606028 non-null  object 
 12  oil          1594963 non-null  float64
 13  wtr          1594963 non-null  float64
 14  days         1594963 non-null  float64
 15  runs         1606028 non-null  int64  
 16  gas          1594963 non-null  float64
 17  gassold      1606028 non-null  int64  
 18  fl

In [7]:
oil['reportdate'] = pd.to_datetime(oil['reportdate'])
oil = oil.set_index('reportdate',drop=False)
oil.index.names = ["date"]
oil['year'] = oil.index.year
oil['month'] = oil.index.month

oil['reportdate']=[str(x)[:-12] for x in oil['reportdate']]

days_in_month = {
    1:31,
    2:28,
    3:31,
    4:30,
    5:31,
    6:30,
    7:31,
    8:31,
    9:30,
    10:31,
    11:30,
    12:31
}
oil['days_in_month'] = oil['month'].map(days_in_month)

In [8]:
# mergers
oil["company"] =oil['company'].str.replace('WPX ENERGY WILLISTON LLC',"DEVON ENERGY", regex=False)
oil["company"] =oil['company'].str.replace('XTO ENERGY INC',"EXXONMOBIL", regex=False)
oil["company"] =oil['company'].str.replace('BURLINGTON RESOURCES OIL & GAS COMPANY LP',"CONOCOPHILLIPS", regex=False)
oil["company"] =oil['company'].str.replace('OASIS PETROLEUM NORTH AMERICA LLC',"CHORD ENERGY", regex=False)
oil["company"] =oil['company'].str.replace('WHITING OIL AND GAS CORPORATION',"CHORD ENERGY", regex=False)

In [9]:
oil['oil_per_day'] = oil['oil']/oil['days_in_month']
oil['gas_per_day'] = oil['gas']/oil['days_in_month']
oil['water_per_day'] = oil['wtr']/oil['days_in_month']
oil['gassold_per_day'] = oil['gassold']/oil['days_in_month']
oil['flared_per_day'] = oil['flared']/oil['days_in_month']
oil['gas_boe'] = oil['gas']/6
oil['gas_boe_per_day'] = oil['gas_per_day']/6
oil['total_boed'] = oil['gas_boe_per_day'] + oil['oil_per_day']

# oil[oil['oil_bbl_da'].isin([np.inf, -np.inf])].index
oil = oil.dropna(0)

C:\Users\matth\AppData\Local\Temp\ipykernel_18184\1573427572.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  oil = oil.dropna(0)


In [10]:
oil.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1594710 entries, 2015-05-01 to 2023-01-01
Data columns (total 33 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   reportdate       1594710 non-null  object 
 1   api_wellno       1594710 non-null  int64  
 2   fileno           1594710 non-null  int64  
 3   company          1594710 non-null  object 
 4   wellname         1594710 non-null  object 
 5   quarter          1594710 non-null  object 
 6   section          1594710 non-null  float64
 7   township         1594710 non-null  float64
 8   range            1594710 non-null  float64
 9   county           1594710 non-null  object 
 10  fieldname        1594710 non-null  object 
 11  pool             1594710 non-null  object 
 12  oil              1594710 non-null  float64
 13  wtr              1594710 non-null  float64
 14  days             1594710 non-null  float64
 15  runs             1594710 non-null  int64  
 16  gas

In [11]:
# oil['company'] = oil['company'].astype('category')
# oil['reportdate'] = oil['reportdate'].astype('category')
# oil['wellname'] = oil['wellname'].astype('category')
# oil['quarter'] = oil['quarter'].astype('category')
# oil['county'] = oil['county'].astype('category')
# oil['county_name'] = oil['county_name'].astype('category')
# oil['fieldname'] = oil['fieldname'].astype('category')
# oil['pool'] = oil['pool'].astype('category')

# oil['fileno'] = oil['fileno'].astype('int32')
# oil['section'] = oil['section'].astype('int8')
# oil['township'] = oil['township'].astype('int16')
# oil['range'] = oil['range'].astype('int16')
# oil['oil'] = oil['oil'].astype('int64')
# oil['gas'] = oil['gas'].astype('int64')
# oil['wtr'] = oil['wtr'].astype('int64')
# oil['month'] = oil['month'].astype('int8')
# oil['days_in_month'] = oil['days_in_month'].astype('int8')

In [12]:
oil.to_pickle("ND_well_data_2015_2023_cleaned.pickle")